In [8]:
import sys
sys.path.append('/home/ranvir/AiCore')
from database_connection import DatabaseConnection
db = DatabaseConnection()

# <img src="https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/12.%20SQL%20GROUP%20BY/images/SQLIcon.png?modified=2223&raw=1" width=80px height=80px style="vertical-align: middle;"> SQL `GROUP BY`

Sometimes instead of aggregating a whole data column, it can be useful to aggregate a **subset** of that data. `GROUP BY` allows you to group records with the same value into one group/set of data, which makes it an indispensable tool to analyse data. Imagine you wanted to check which of your customers were the best and had rented the most movies from you. We could `SUM` the number of rows which exist for that customer but it would take a long time.

We can instead use `GROUP BY` to group the table into different groups of data based on their value. Aggregations are normally applied to the remaining columns to aggregate the columns within each group.

The syntax for `GROUP BY` is:

In [ ]:
SELECT {column}, {aggregation1, aggregation2 ...}
FROM {table}
GROUP BY {column}

Let's look at the example of grouping customers together in the `rental` table and counting the amount of movies they have rented. We could do this with the following query, returning the top five customers in descending order:

In [9]:
db.execute_query('''
                    SELECT customer_id,
                           COUNT(rental_date) AS total_number_of_rentals
                    FROM
                        rental
                    GROUP BY
                        customer_id
                    ORDER BY
                        total_number_of_rentals DESC
                    LIMIT
                        5;
                ''')

,customer_id,total_number_of_rentals
0,148,46
1,526,45
2,236,42
3,144,42
4,75,41


We are using the `GROUP BY` to group the records by `customer_id` so wherever the `customer_id`'s are the same, they will be grouped together in the same row. We've also selected another column as an aggregation `COUNT(rental_date) AS total_number_of_rentals` so that within each group the number of rentals are counted for each customer.

The results are then ordered by `total_number_of_rentals` in descending order, such that users with the highest amount of rentals are shown first. Finally the records are then limited to the top five users with the `LIMIT 5` statement.

>In the following and preceding images the coloured row overlay was added to the images as an aid to display the grouping SQL is performing.

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/12.%20SQL%20GROUP%20BY/images/groupby_sql.png?modified=3423324&raw=1' >

From the image above, we can observe that our query is grouping all the green columns together with the `GROUP BY customer_id` statement and then counting all values in the `rental_date` with `COUNT(rental_date) AS total_number_of_rentals` to get the number of rentals a customer has made. Similarly all values in the red rows will be grouped together and any additional values in the table (not shown in the image) where `customer_id` is equal to `211`. The same will happen with all the yellow rows and any additional rows where `customer_id` is equal to `209`.

Multiple columns can also be added to the `GROUP BY` column to further group the data if required. Let's imagine we want to check, how many movies each staff member sold to a customer. We could first use a `GROUP BY` to group the data by the `customer_id` and then group by a secondary column `staff_id` to further group the data. The query is:

In [16]:
db.execute_query('''
                    SELECT customer_id,
                           COUNT(rental_date) AS total_number_of_rentals,
                           staff_id
                    FROM
                        rental
                    GROUP BY
                        customer_id, staff_id
                    ORDER BY
                        customer_id DESC, staff_id DESC;
                ''')

,customer_id,total_number_of_rentals,staff_id
0,599,8,2
1,599,11,1
2,598,13,2
3,598,9,1
4,597,15,2
...,...,...,...
1193,3,12,1
1194,2,12,2
1195,2,15,1
1196,1,17,2


The query is saying `GROUP BY customer_id, staff_id` which is the equivalent of saying, in all rows where `customer_id` and `staff_id` match then group those rows together, and count all the rows in the `rental_date` column.

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/12.%20SQL%20GROUP%20BY/images/double_group_by.png?modified=231232&raw=1'>

For the rows in red, green and yellow in the image above, you can see that the `customer_id` and `staff_id` match so they will be grouped together by the `GROUP BY` clause. Those rows are then counted for each group with the `COUNT(rental_date) AS total_number_of_rentals` statement. Before performing your `GROUP BY` statement it can be helpful to first look at the data in the table. This can help to visualise what rows will be grouped together before writing the statement.

A common error you might run into when trying to perform `GROUP BY` statements is:

`column x must appear in the GROUP BY clause or be used in an aggregate function`

Where `x` is one of the columns in your `SELECT` statement, why does this happen? Let's try and use `SELECT` to also select the `inventory_id` column in our statement and investigate what SQL is trying to do:

In [17]:
db.execute_query('''
                    SELECT customer_id,
                           COUNT(rental_date) AS total_number_of_rentals,
                           staff_id,
                           inventory_id
                    FROM
                        rental
                    GROUP BY
                        customer_id, staff_id
                    ORDER BY
                        customer_id DESC, staff_id DESC;
                ''')

ProgrammingError: (psycopg2.errors.GroupingError) column "rental.inventory_id" must appear in the GROUP BY clause or be used in an aggregate function
LINE 5:                         inventory_id
                                ^

[SQL: 
                    SELECT customer_id,
                        COUNT(rental_date) AS total_number_of_rentals,
                        staff_id,
                        inventory_id
                    FROM
                        rental
                    GROUP BY
                        customer_id, staff_id
                    ORDER BY
                        customer_id DESC, staff_id DESC;
                ]
(Background on this error at: https://sqlalche.me/e/20/f405)

When we try to run this query, the error returned is:

`column "rental.inventory_id" must appear in the GROUP BY clause or be used in an aggregate function`

Let's check which rows SQL is trying to group together to see why this might be the case:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/12.%20SQL%20GROUP%20BY/images/group_by_errors.png?modified=323423&raw=1'>

The rows of each colour match our `GROUP BY customer_id, staff_id` since in each of those row the two values are the same. Remember SQL will try and summarise these rows into one row since that was what was specified in our `GROUP BY`.

If you notice with each grouping there are many different values for the value of `inventory_id` this is the complaint SQL is making. SQL doesn't know how to `SELECT` from the `inventory_id` column since there are many different possible choices of `inventory_id`, which one should it choose? We can rectify this by following SQL's advice:

`column "rental.inventory_id" must appear in the GROUP BY clause or be used in an aggregate function`

So we either need to aggregate that column or add the column to our `GROUP BY` clause. Let's add the `COUNT` of the `inventory_id` for each of our groups:

In [21]:
db.execute_query('''
                    SELECT customer_id,
                           COUNT(rental_date) AS total_number_of_rentals,
                           staff_id,
                           COUNT(inventory_id) AS number_of_unique_rentals
                    FROM
                        rental
                    GROUP BY
                        customer_id, staff_id
                    ORDER BY
                        customer_id DESC, staff_id DESC;
                ''')

,customer_id,total_number_of_rentals,staff_id,number_of_unique_rentals
0,599,8,2,8
1,599,11,1,11
2,598,13,2,13
3,598,9,1,9
4,597,15,2,15
...,...,...,...,...
1193,3,12,1,12
1194,2,12,2,12
1195,2,15,1,15
1196,1,17,2,17


Great, that solves our problem but might not be particularly useful since it should just return the same `COUNT` as the `rental_date`. What about if we add it to the `GROUP BY`:

In [20]:
db.execute_query('''
                    SELECT customer_id,
                           COUNT(rental_date) AS total_number_of_rentals,
                           staff_id,
                           COUNT(inventory_id) AS number_of_unique_rentals
                    FROM
                        rental
                    GROUP BY
                        customer_id, staff_id, inventory_id
                    ORDER BY
                        customer_id DESC, staff_id DESC;                
                ''')

,customer_id,total_number_of_rentals,staff_id,number_of_unique_rentals
0,599,1,2,1
1,599,1,2,1
2,599,1,2,1
3,599,1,2,1
4,599,1,2,1
...,...,...,...,...
16039,1,1,1,1
16040,1,1,1,1
16041,1,1,1,1
16042,1,1,1,1


This give us something a little more useable, it returns the amount of times an item was rented by a particular customer from a particular staff member.

Just be aware of the possibility of this error happening, it can be common. If it happens, it can be useful to visualise data again to check what rows you're grouping together.

## `HAVING`

In the SQL order of execution the `WHERE` statement is performed filtering the data before the `GROUP BY` is applied. What if we want to filter the data after the `GROUP BY`?. We can apply a `HAVING` clause after the `GROUP BY`. `HAVING` comes after `GROUP BY` in the order of execution. This allows us to filter the data so that we only get results we are interested in.

The syntax for the `HAVING` statement is as follows:


In [ ]:
HAVING {conditional}

The `HAVING` statement takes a conditional and if the result of the conditional evaluates to `True`, then the rows will become part of the returned query set. We can apply it in a statement like so:

In [22]:
db.execute_query('''
                    SELECT customer_id,
                           COUNT(rental_date) AS total_number_of_rentals
                    FROM
                        rental
                    GROUP BY
                        customer_id
                    HAVING
                        COUNT(rental_date) > 37
                    ORDER BY
                        total_number_of_rentals DESC;
                ''')


,customer_id,total_number_of_rentals
0,148,46
1,526,45
2,236,42
3,144,42
4,75,41
5,197,40
6,469,40
7,178,39
8,468,39
9,137,39


This only return the rows where the count of rentals for that customer is greater than `37`. You can apply the same conditions you would apply in a `WHERE` statement. The important distinction is that `WHERE` is applied before the `GROUP BY` and `HAVING` is applied after the `GROUP BY`. Let's look at a more complex example including both:

In [24]:
db.execute_query('''
                    SELECT SUM(rental_rate * rental_duration) AS total_rating_rental_rate, rating
                    FROM film
                    WHERE rental_rate > 0.99
                    GROUP BY rating
                    HAVING SUM(rental_rate * rental_duration) > 2500;
                ''')


,total_rating_rental_rate,rating
0,3102.20,PG-13
1,2809.02,NC-17
2,2780.09,PG


In this query we've aggregated the data so that we can check the potential value of rentals for each film rating which total over `2500`.

First we apply the statement `WHERE rental_rate > 0.99` this allows us to focus only on non-budget movies. We then perform the `GROUP BY` on the `rating` column to group the movies into their respective ratings. We then `SELECT` the `SUM` of of the total `rental_rate` and `rental_duration` to get the total possible value of the rentals in that `rating` category. We finally apply our `HAVING` statement to only display film rating having a value of over 2500.

>Remember the `WHERE` and `HAVING` clauses both filter but at different stages of the **SQL order of execution**:
>
>- `WHERE` is pre-filtering the data before the grouping
>- `HAVING` is filtering the data after the grouping has taken place.

## Key Takeaways

- `GROUP BY` can be powerful when you want to perform analysis on groups of your data
- You can use **aggregations** in your `SELECT` statement with a `GROUP BY` to help your data tell a story
- Since you will be performing aggregations with the `GROUP BY` statement remember to **alias** your columns so the resultant data makes sense
- It can be useful to perform on `SELECT` statement on the data first so you can view which groups you want to perform a `GROUP BY` on
- Remember that columns selected in your `SELECT` statement need to either be included in the `GROUP BY` statement or **aggregated** in the `SELECT` statement
- You can use the `HAVING` statement to perform filtering after the `GROUP BY` has taken place
